# Quantum Reservoir Computing for Electricity Grid Load Forecasting

## Abstract

Short-term electricity load forecasting is essential for stable and efficient grid operation. While classical machine learning models achieve strong performance, they often require large numbers of trainable parameters to capture nonlinear temporal dynamics.

This notebook implements a hybrid quantum–classical forecasting pipeline based on **Quantum Reservoir Computing (QRC)**, using half-hourly UK electricity grid data. A continuous-variable quantum reservoir transforms classical time-series inputs into a high-dimensional nonlinear feature space, followed by a lightweight classical readout layer. The model is evaluated against established classical baselines to assess the potential of quantum-enhanced time-series prediction. 


## Authorship and Project Credit

This notebook is derived from a collaborative research project conducted.

Contributing authors:

* Dominic Kosnica
* Sennai Kahssai 
* Dawinder Bansrow 
* Michael Liu

All collaborators contributed to data processing, modeling, and analysis. This notebook represents a curated and extended version of the original group project for research portfolio purposes.


## Relationship to Classical Baseline Model

This quantum forecasting pipeline is the direct continuation of a classical benchmark model developed in the companion ML-AI repository:

**Classical_Baseline_Model_for_Electricity_Grid_Load_Forecasting.ipynb**

That notebook establishes strong classical performance using Random Forest, Ridge Regression, and Naive baselines on the same UK electricity grid dataset.

The present work evaluates whether Quantum Reservoir Computing can match or exceed this classical benchmark under identical data and preprocessing conditions. This classical–quantum pairing ensures scientifically controlled comparison rather than isolated model demonstration.


## 1. Motivation for Quantum-Enhanced Forecasting

Classical machine learning models can achieve high accuracy in electricity load forecasting but often require extensive training and large parameter counts to model nonlinear temporal structure, weather interactions, and seasonal effects.

Quantum reservoir computing offers an alternative paradigm. Instead of training deep networks, a fixed quantum dynamical system performs nonlinear feature expansion, and only a simple classical readout layer is trained. In principle, quantum dynamics can generate expressive feature maps with fewer trainable parameters, offering a potential path toward more efficient learning as quantum hardware matures. 


## 2. Choice of Quantum Reservoir Computing Approach

We adopt **Continuous-Variable Quantum Reservoir Computing (CV-QRC)** as the primary model architecture. CV-QRC is well suited for real-valued time-series data and can encode continuous classical inputs directly into quantum states without requiring large qubit registers.

Unlike qubit-based QRC, which relies on discrete two-level systems, CV-QRC uses bosonic modes and naturally models continuous quantities such as amplitudes and phases. This makes it a practical candidate for near-term quantum devices and particularly appropriate for electricity load data, which is inherently continuous. 


## 3. Data Processing and Feature Encoding

Half-hourly electricity load data is converted into a supervised learning format using lagged demand features. Cyclical temporal variables are encoded using sine and cosine transformations to preserve periodic structure without artificial discontinuities.

These steps transform the raw time series into a structured input matrix suitable for injection into the quantum reservoir at each time step. 


## 4. Quantum Reservoir Architecture

The model employs a fixed continuous-variable quantum circuit as a reservoir. At each time step, classical input features modulate parameters such as displacement or rotation operators. The quantum state evolves under fixed reservoir dynamics, generating a nonlinear transformation of the input.

Expectation values of selected observables (e.g., quadrature measurements) are extracted to form the quantum feature vector. A classical linear readout layer is then trained to predict the next-step electricity demand. 


## 5. Comparison With Classical Forecasting Models

The quantum reservoir model is evaluated on the same dataset and forecasting task as the classical baselines, including naive persistence, ridge regression, and random forest models.

This controlled comparison ensures that any observed performance differences reflect modeling methodology rather than dataset or preprocessing variation. The classical random forest model establishes a strong benchmark that the quantum model must match or exceed in accuracy or efficiency. 


## 6. Limitations

This implementation evaluates the quantum reservoir in simulation, which does not fully capture the effects of hardware noise and loss present in real photonic systems. In addition, near-term devices limit the number of available quantum modes, constraining input dimensionality.

Computational cost also grows rapidly with reservoir size in simulation, limiting scalability in current experiments. 


## 7. Future Directions

Future work will focus on:

* Transitioning from simulation to photonic hardware implementations of CV-QRC
* Integrating hybrid quantum reservoirs with attention-based temporal models
* Scaling reservoirs under fault-tolerant continuous-variable quantum computing

These extensions aim to evaluate whether quantum reservoirs can provide practical advantages for large-scale time-series forecasting as hardware capabilities improve. 


## 8. Broader Impact

Improving short-term electricity load forecasting supports grid reliability, renewable energy integration, and efficient system operation. More accurate predictions reduce reserve requirements and operating costs while enhancing system resilience.

Quantum-enhanced forecasting methods offer a potential future pathway to scalable nonlinear modeling with reduced training complexity, contributing to cleaner, more reliable energy systems over time. 


## 9. Reproducibility

All experiments are fully reproducible. Data preprocessing, model construction, training, and evaluation are executed deterministically within this notebook.

The pipeline is designed to mirror the classical forecasting workflow used in the ML-AI repository to ensure fair benchmarking. This enables transparent replication and extension of results as quantum hardware and simulators evolve.


In [ ]:
# Optional: install dependencies in a fresh runtime (e.g., Colab)
# !pip install -q pennylane strawberryfields pennylane-sf scipy==1.10.1 scikit-learn pandas openpyxl qiskit


## Small QRC (9 qubits, 1–3 body readout, 387 features)

In [16]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Pauli




FILEPATH = r"C:/Users/domin/OneDrive/Documents/QUAC 605 Project Data.xlsx"

np.random.seed(42)

NUM_QUBITS    = 9      
SIGMA         = 0.1     
DEPTH         = 1     
WINDOW        = 4     
ANGLE_SCALE   = 0.1     
KERR_STRENGTH = 0.01     


# 1. LOAD AND PREPARE RAW DATA (CLASSICAL)


df = pd.read_excel(FILEPATH)

print("Columns in df:", df.columns.tolist())
print("Number of rows before cleaning:", len(df))


max_period = 48
df["period_sin"] = np.sin(2 * np.pi * df["SETTLEMENT_PERIOD"] / max_period)
df["period_cos"] = np.cos(2 * np.pi * df["SETTLEMENT_PERIOD"] / max_period)

feature_cols = [
    "EMBEDDED_WIND_GENERATION",   
    "EMBEDDED_SOLAR_GENERATION",  
    "PUMP_STORAGE_PUMPING",       
    "SCOTTISH_TRANSFER",          
    "NEMO_FLOW",                  
    "ELECLINK_FLOW",              
    "Temperature",                
    "period_sin",                 
    "period_cos",                 
]

target_col = "ENGLAND_WALES_DEMAND"


df = df.dropna(subset=feature_cols + [target_col]).reset_index(drop=True)
print("Number of rows after dropna:", len(df))


X_raw = df[feature_cols].values.astype(float)   
y_raw = df[target_col].values.astype(float)     

print("X_raw shape:", X_raw.shape)
print("y_raw shape:", y_raw.shape)


# 2. SCALE INPUT FEATURES AND TARGET (CLASSICAL)


X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(X_raw)
print("X_scaled shape:", X_scaled.shape)

y_scaler = StandardScaler()
y_scaled = y_scaler.fit_transform(y_raw.reshape(-1, 1)).ravel()
print("y_scaled shape:", y_scaled.shape)

T_total, n_features = X_scaled.shape


# 3. BUILD SLIDING WINDOWS (CLASSICAL)


X_windows = []         
y_targets_scaled = []   
y_targets_raw    = []   

for t in range(WINDOW, T_total):
    window = X_scaled[t - WINDOW : t]      
    X_windows.append(window)
    y_targets_scaled.append(y_scaled[t])   
    y_targets_raw.append(y_raw[t])         

X_windows        = np.array(X_windows)          
y_targets_scaled = np.array(y_targets_scaled)   
y_targets_raw    = np.array(y_targets_raw)      

num_samples = X_windows.shape[0]

print("X_windows shape       :", X_windows.shape)
print("y_targets_scaled shape:", y_targets_scaled.shape)
print("y_targets_raw shape   :", y_targets_raw.shape)

assert num_samples == y_targets_scaled.shape[0] == y_targets_raw.shape[0], \
    "Window/target shapes inconsistent!"


# 4. BUILD FIXED LINEAR RESERVOIR (GAUSSIAN UNITARY, SHALLOW)


def build_gaussian_unitary(num_qubits=NUM_QUBITS, depth=DEPTH, sigma=SIGMA):
    
    
    qc = QuantumCircuit(num_qubits)
    for _ in range(depth):
        
        for q in range(num_qubits):
            qc.rx(np.random.normal(0, sigma), q)
            qc.ry(np.random.normal(0, sigma), q)
            qc.rz(np.random.normal(0, sigma), q)
        
        for q in range(num_qubits - 1):
            qc.cz(q, q + 1)
    return qc.to_gate(label="U_linear")

U_linear = build_gaussian_unitary()
print("Built linear (Gaussian) unitary reservoir with DEPTH =", DEPTH)


# 5. PRE-BUILD PAULI OPERATORS FOR READOUT (ALL-PAIR + TRIPLES)


def build_pauli_operators(num_qubits):
    
    single_X = []
    single_Y = []
    single_Z = []
    pair_ZZ  = []
    pair_XX  = []
    pair_YY  = []
    triple_ZZZ = []
    triple_XXX = []
    triple_YYY = []

    
    for i in range(num_qubits):
        
        sX = "".join("X" if j == i else "I" for j in range(num_qubits))
        
        sY = "".join("Y" if j == i else "I" for j in range(num_qubits))
        
        sZ = "".join("Z" if j == i else "I" for j in range(num_qubits))

        single_X.append(Pauli(sX))
        single_Y.append(Pauli(sY))
        single_Z.append(Pauli(sZ))

    
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            sZZ = ""
            sXX = ""
            sYY = ""
            for k in range(num_qubits):
                if k == i or k == j:
                    sZZ += "Z"
                    sXX += "X"
                    sYY += "Y"
                else:
                    sZZ += "I"
                    sXX += "I"
                    sYY += "I"
            pair_ZZ.append(Pauli(sZZ))
            pair_XX.append(Pauli(sXX))
            pair_YY.append(Pauli(sYY))

    
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            for k in range(j + 1, num_qubits):
                sZZZ = ""
                sXXX = ""
                sYYY = ""
                for idx in range(num_qubits):
                    if idx == i or idx == j or idx == k:
                        sZZZ += "Z"
                        sXXX += "X"
                        sYYY += "Y"
                    else:
                        sZZZ += "I"
                        sXXX += "I"
                        sYYY += "I"
                triple_ZZZ.append(Pauli(sZZZ))
                triple_XXX.append(Pauli(sXXX))
                triple_YYY.append(Pauli(sYYY))

    return (
        single_X, single_Y, single_Z,
        pair_ZZ, pair_XX, pair_YY,
        triple_ZZZ, triple_XXX, triple_YYY
    )

(
    PAULI_X, PAULI_Y, PAULI_Z,
    PAULI_ZZ, PAULI_XX, PAULI_YY,
    PAULI_ZZZ, PAULI_XXX3, PAULI_YYY3
) = build_pauli_operators(NUM_QUBITS)

print("Pauli readout operators built:")
print("  single X:", len(PAULI_X))
print("  single Y:", len(PAULI_Y))
print("  single Z:", len(PAULI_Z))
print("  ZZ pairs (all-pair):", len(PAULI_ZZ))
print("  XX pairs (all-pair):", len(PAULI_XX))
print("  YY pairs (all-pair):", len(PAULI_YY))
print("  ZZZ triples (all-triple):", len(PAULI_ZZZ))
print("  XXX triples (all-triple):", len(PAULI_XXX3))
print("  YYY triples (all-triple):", len(PAULI_YYY3))

# Total feature count:
# single: 3 * NUM_QUBITS
# pairs:  3 * C(NUM_QUBITS, 2)
# triples:3 * C(NUM_QUBITS, 3)
num_pairs   = NUM_QUBITS * (NUM_QUBITS - 1) // 2
num_triples = NUM_QUBITS * (NUM_QUBITS - 1) * (NUM_QUBITS - 2) // 6
num_features_qrc = 3 * NUM_QUBITS + 3 * num_pairs + 3 * num_triples
print("Total QRC features per window:", num_features_qrc)


# 6. KERR-QRC FEATURE MAP (QUANTUM)


def apply_kerr_layer(qc, x, kerr_strength=KERR_STRENGTH):
    
    for q in range(NUM_QUBITS):
        theta_q = ANGLE_SCALE * x[q]
        kerr_angle = kerr_strength * (theta_q ** 2)
        qc.rz(kerr_angle, q)

def reservoir_features_kerr(input_window):
    
    T, F = input_window.shape
    qc = QuantumCircuit(NUM_QUBITS)

    for t in range(T):
        x = input_window[t]

        
        for q in range(NUM_QUBITS):
            qc.ry(ANGLE_SCALE * x[q], q)

        
        apply_kerr_layer(qc, x, kerr_strength=KERR_STRENGTH)

        
        theta_global = ANGLE_SCALE * x[8]
        for q in range(NUM_QUBITS):
            qc.rz(theta_global, q)

        
        qc.append(U_linear, range(NUM_QUBITS))

    # Final quantum state
    state = Statevector.from_instruction(qc)

    feats = []

    
    for p in PAULI_X:
        feats.append(np.real(state.expectation_value(p)))

    
    for p in PAULI_Y:
        feats.append(np.real(state.expectation_value(p)))

    
    for p in PAULI_Z:
        feats.append(np.real(state.expectation_value(p)))

    
    for p in PAULI_ZZ:
        feats.append(np.real(state.expectation_value(p)))

    
    for p in PAULI_XX:
        feats.append(np.real(state.expectation_value(p)))

    
    for p in PAULI_YY:
        feats.append(np.real(state.expectation_value(p)))

    
    for p in PAULI_ZZZ:
        feats.append(np.real(state.expectation_value(p)))

    
    for p in PAULI_XXX3:
        feats.append(np.real(state.expectation_value(p)))

    
    for p in PAULI_YYY3:
        feats.append(np.real(state.expectation_value(p)))

    return np.array(feats, dtype=float)

def build_reservoir_features_for_set_kerr(X_windows_scaled):
    
    num_samples_local = X_windows_scaled.shape[0]
    feat_list = []
    for idx, window in enumerate(X_windows_scaled):
        feat_list.append(reservoir_features_kerr(window))
        if (idx + 1) % 10 == 0:
            print(f"Processed {idx+1}/{num_samples_local} windows...", end="\r")
    print()
    return np.vstack(feat_list)


# 7. BUILD KERR-QRC FEATURE MATRIX


print("\nBuilding Kerr-QRC feature vectors for ALL windows (all-pair + all-triple)...")
QRC_features = build_reservoir_features_for_set_kerr(X_windows)
print("QRC_features shape:", QRC_features.shape)

assert QRC_features.shape[0] == num_samples, "QRC_features and y_targets mismatch!"

print("\nFeature means (first 5):", QRC_features.mean(axis=0)[:5])
print("Feature stds  (first 5):", QRC_features.std(axis=0)[:5])


# 8. TRAIN/TEST SPLIT FOR CLASSICAL READOUT


H = QRC_features  

train_size = int(0.8 * num_samples)

X_train = H[:train_size]
X_test  = H[train_size:]

y_train_scaled = y_targets_scaled[:train_size]  
y_test_scaled  = y_targets_scaled[train_size:]   
y_train_raw    = y_targets_raw[:train_size]      
y_test_raw     = y_targets_raw[train_size:]      

print("\nTrain/Test shapes:")
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train_scaled:", y_train_scaled.shape)
print("y_test_scaled :", y_test_scaled.shape)
print("y_train_raw   :", y_train_raw.shape)
print("y_test_raw    :", y_test_raw.shape)


Columns in df: ['SETTLEMENT_PERIOD', 'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION', 'EMBEDDED_SOLAR_GENERATION', 'PUMP_STORAGE_PUMPING', 'SCOTTISH_TRANSFER', 'NEMO_FLOW', 'ELECLINK_FLOW', 'Temperature', 'period_sin', 'period_cos']
Number of rows before cleaning: 1488
Number of rows after dropna: 1488
X_raw shape: (1488, 9)
y_raw shape: (1488,)
X_scaled shape: (1488, 9)
y_scaled shape: (1488,)
X_windows shape       : (1484, 4, 9)
y_targets_scaled shape: (1484,)
y_targets_raw shape   : (1484,)
Built linear (Gaussian) unitary reservoir with DEPTH = 1
Pauli readout operators built:
  single X: 9
  single Y: 9
  single Z: 9
  ZZ pairs (all-pair): 36
  XX pairs (all-pair): 36
  YY pairs (all-pair): 36
  ZZZ triples (all-triple): 84
  XXX triples (all-triple): 84
  YYY triples (all-triple): 84
Total QRC features per window: 387

Building Kerr-QRC feature vectors for ALL windows (all-pair + all-triple)...
Processed 1480/1484 windows...
QRC_features shape: (1484, 387)

Feature means (first

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


# RANDOM FOREST READOUT FOR KERR QUBIT QRC (USE QRC_features DIRECTLY)



num_samples_qb = QRC_features.shape[0]


assert len(y_targets_raw) == num_samples_qb, "Qubit-QRC: y_targets_raw and QRC_features misaligned!"

print("\nQubit QRC feature shape:", QRC_features.shape)


train_size_qb = int(0.8 * num_samples_qb)

X_train_qb = QRC_features[:train_size_qb]
X_test_qb  = QRC_features[train_size_qb:]

y_train_qb = y_targets_raw[:train_size_qb]
y_test_qb  = y_targets_raw[train_size_qb:]

print("\n[Qubit-QRC RF] Train/Test shapes:")
print("X_train_qb:", X_train_qb.shape)
print("X_test_qb :", X_test_qb.shape)
print("y_train_qb:", y_train_qb.shape)
print("y_test_qb :", y_test_qb.shape)

rf_qb = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    random_state=0,
    n_jobs=-1
)

rf_qb.fit(X_train_qb, y_train_qb)
y_pred_qb = rf_qb.predict(X_test_qb)

mae_qb  = mean_absolute_error(y_test_qb, y_pred_qb)
rmse_qb = np.sqrt(mean_squared_error(y_test_qb, y_pred_qb))
r2_qb   = r2_score(y_test_qb, y_pred_qb)

print("\n===== RANDOM FOREST READOUT (KERR QUBIT QRC) =====")
print(f"MAE  (MW) : {mae_qb:,.3f}")
print(f"RMSE (MW) : {rmse_qb:,.3f}")
print(f"R^2       : {r2_qb:,.4f}")



Qubit QRC feature shape: (1484, 387)

[Qubit-QRC RF] Train/Test shapes:
X_train_qb: (1187, 387)
X_test_qb : (297, 387)
y_train_qb: (1187,)
y_test_qb : (297,)

===== RANDOM FOREST READOUT (KERR QUBIT QRC) =====
MAE  (MW) : 1,016.593
RMSE (MW) : 1,323.688
R^2       : 0.9358


In [19]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Pauli

FILEPATH = r"C:/Users/domin/OneDrive/Documents/QUAC 605 Project Data.xlsx"

np.random.seed(42)

NUM_QUBITS    = 9
SIGMA         = 0.1
DEPTH         = 1
WINDOW        = 8
ANGLE_SCALE   = 0.1
KERR_STRENGTH = 0.1

# 1. LOAD AND PREPARE RAW DATA (CLASSICAL)

df = pd.read_excel(FILEPATH)

print("Columns in df:", df.columns.tolist())
print("Number of rows before cleaning:", len(df))

max_period = 48
df["period_sin"] = np.sin(2 * np.pi * df["SETTLEMENT_PERIOD"] / max_period)
df["period_cos"] = np.cos(2 * np.pi * df["SETTLEMENT_PERIOD"] / max_period)

feature_cols = [
    "EMBEDDED_WIND_GENERATION",
    "EMBEDDED_SOLAR_GENERATION",
    "PUMP_STORAGE_PUMPING",
    "SCOTTISH_TRANSFER",
    "NEMO_FLOW",
    "ELECLINK_FLOW",
    "Temperature",
    "period_sin",
    "period_cos",
]

target_col = "ENGLAND_WALES_DEMAND"

df = df.dropna(subset=feature_cols + [target_col]).reset_index(drop=True)
print("Number of rows after dropna:", len(df))

X_raw = df[feature_cols].values.astype(float)
y_raw = df[target_col].values.astype(float)

print("X_raw shape:", X_raw.shape)
print("y_raw shape:", y_raw.shape)

# 2. SCALE INPUT FEATURES AND TARGET (CLASSICAL)

X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(X_raw)
print("X_scaled shape:", X_scaled.shape)

y_scaler = StandardScaler()
y_scaled = y_scaler.fit_transform(y_raw.reshape(-1, 1)).ravel()
print("y_scaled shape:", y_scaled.shape)

T_total, n_features = X_scaled.shape

# 3. BUILD SLIDING WINDOWS (CLASSICAL)

X_windows = []
y_targets_scaled = []
y_targets_raw    = []

for t in range(WINDOW, T_total):
    window = X_scaled[t - WINDOW : t]
    X_windows.append(window)
    y_targets_scaled.append(y_scaled[t])
    y_targets_raw.append(y_raw[t])

X_windows        = np.array(X_windows)
y_targets_scaled = np.array(y_targets_scaled)
y_targets_raw    = np.array(y_targets_raw)

num_samples = X_windows.shape[0]

print("X_windows shape       :", X_windows.shape)
print("y_targets_scaled shape:", y_targets_scaled.shape)
print("y_targets_raw shape   :", y_targets_raw.shape)

assert num_samples == y_targets_scaled.shape[0] == y_targets_raw.shape[0]

# 4. BUILD FIXED LINEAR RESERVOIR

def build_gaussian_unitary(num_qubits=NUM_QUBITS, depth=DEPTH, sigma=SIGMA):
    qc = QuantumCircuit(num_qubits)
    for _ in range(depth):
        for q in range(num_qubits):
            qc.rx(np.random.normal(0, sigma), q)
            qc.ry(np.random.normal(0, sigma), q)
            qc.rz(np.random.normal(0, sigma), q)
        for q in range(num_qubits - 1):
            qc.cz(q, q + 1)
    return qc.to_gate(label="U_linear")

U_linear = build_gaussian_unitary()
print("Built linear (Gaussian) unitary reservoir with DEPTH =", DEPTH)

# 5. PRE-BUILD PAULI OPERATORS FOR READOUT

def build_pauli_operators(num_qubits):
    single_X = []
    single_Y = []
    single_Z = []
    pair_ZZ  = []
    pair_XX  = []
    pair_YY  = []
    triple_ZZZ = []
    triple_XXX = []
    triple_YYY = []
    quad_ZZZZ  = []
    quad_XXXX  = []
    quad_YYYY  = []
    quint_ZZZZZ = []
    quint_XXXXX = []
    quint_YYYYY = []
    sext_ZZZZZZ = []
    sext_XXXXXX = []
    sext_YYYYYY = []

    for i in range(num_qubits):
        sX = "".join("X" if j == i else "I" for j in range(num_qubits))
        sY = "".join("Y" if j == i else "I" for j in range(num_qubits))
        sZ = "".join("Z" if j == i else "I" for j in range(num_qubits))
        single_X.append(Pauli(sX))
        single_Y.append(Pauli(sY))
        single_Z.append(Pauli(sZ))

    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            sZZ = ""
            sXX = ""
            sYY = ""
            for k in range(num_qubits):
                if k == i or k == j:
                    sZZ += "Z"
                    sXX += "X"
                    sYY += "Y"
                else:
                    sZZ += "I"
                    sXX += "I"
                    sYY += "I"
            pair_ZZ.append(Pauli(sZZ))
            pair_XX.append(Pauli(sXX))
            pair_YY.append(Pauli(sYY))

    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            for k in range(j + 1, num_qubits):
                sZZZ = ""
                sXXX = ""
                sYYY = ""
                for idx in range(num_qubits):
                    if idx == i or idx == j or idx == k:
                        sZZZ += "Z"
                        sXXX += "X"
                        sYYY += "Y"
                    else:
                        sZZZ += "I"
                        sXXX += "I"
                        sYYY += "I"
                triple_ZZZ.append(Pauli(sZZZ))
                triple_XXX.append(Pauli(sXXX))
                triple_YYY.append(Pauli(sYYY))

    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            for k in range(j + 1, num_qubits):
                for l in range(k + 1, num_qubits):
                    sZZZZ = ""
                    sXXXX = ""
                    sYYYY = ""
                    for idx in range(num_qubits):
                        if idx in (i, j, k, l):
                            sZZZZ += "Z"
                            sXXXX += "X"
                            sYYYY += "Y"
                        else:
                            sZZZZ += "I"
                            sXXXX += "I"
                            sYYYY += "I"
                    quad_ZZZZ.append(Pauli(sZZZZ))
                    quad_XXXX.append(Pauli(sXXXX))
                    quad_YYYY.append(Pauli(sYYYY))

    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            for k in range(j + 1, num_qubits):
                for l in range(k + 1, num_qubits):
                    for m in range(l + 1, num_qubits):
                        sZZZZZ = ""
                        sXXXXX = ""
                        sYYYYY = ""
                        for idx in range(num_qubits):
                            if idx in (i, j, k, l, m):
                                sZZZZZ += "Z"
                                sXXXXX += "X"
                                sYYYYY += "Y"
                            else:
                                sZZZZZ += "I"
                                sXXXXX += "I"
                                sYYYYY += "I"
                        quint_ZZZZZ.append(Pauli(sZZZZZ))
                        quint_XXXXX.append(Pauli(sXXXXX))
                        quint_YYYYY.append(Pauli(sYYYYY))

    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            for k in range(j + 1, num_qubits):
                for l in range(k + 1, num_qubits):
                    for m in range(l + 1, num_qubits):
                        for n in range(m + 1, num_qubits):
                            sZZZZZZ = ""
                            sXXXXXX = ""
                            sYYYYYY = ""
                            for idx in range(num_qubits):
                                if idx in (i, j, k, l, m, n):
                                    sZZZZZZ += "Z"
                                    sXXXXXX += "X"
                                    sYYYYYY += "Y"
                                else:
                                    sZZZZZZ += "I"
                                    sXXXXXX += "I"
                                    sYYYYYY += "I"
                            sext_ZZZZZZ.append(Pauli(sZZZZZZ))
                            sext_XXXXXX.append(Pauli(sXXXXXX))
                            sext_YYYYYY.append(Pauli(sYYYYYY))

    return (
        single_X, single_Y, single_Z,
        pair_ZZ, pair_XX, pair_YY,
        triple_ZZZ, triple_XXX, triple_YYY,
        quad_ZZZZ, quad_XXXX, quad_YYYY,
        quint_ZZZZZ, quint_XXXXX, quint_YYYYY,
        sext_ZZZZZZ, sext_XXXXXX, sext_YYYYYY
    )

(
    PAULI_X, PAULI_Y, PAULI_Z,
    PAULI_ZZ, PAULI_XX, PAULI_YY,
    PAULI_ZZZ, PAULI_XXX3, PAULI_YYY3,
    PAULI_ZZZZ4, PAULI_XXXX4, PAULI_YYYY4,
    PAULI_ZZZZZ5, PAULI_XXXXX5, PAULI_YYYYY5,
    PAULI_ZZZZZZ6, PAULI_XXXXXX6, PAULI_YYYYYY6
) = build_pauli_operators(NUM_QUBITS)

print("Pauli readout operators built:")
print("  single X:", len(PAULI_X))
print("  single Y:", len(PAULI_Y))
print("  single Z:", len(PAULI_Z))
print("  ZZ pairs (all-pair):", len(PAULI_ZZ))
print("  XX pairs (all-pair):", len(PAULI_XX))
print("  YY pairs (all-pair):", len(PAULI_YY))
print("  ZZZ triples (all-triple):", len(PAULI_ZZZ))
print("  XXX triples (all-triple):", len(PAULI_XXX3))
print("  YYY triples (all-triple):", len(PAULI_YYY3))
print("  ZZZZ quads  (all-quad):", len(PAULI_ZZZZ4))
print("  XXXX quads  (all-quad):", len(PAULI_XXXX4))
print("  YYYY quads  (all-quad):", len(PAULI_YYYY4))
print("  ZZZZZ quints (all-quint):", len(PAULI_ZZZZZ5))
print("  XXXXX quints (all-quint):", len(PAULI_XXXXX5))
print("  YYYYY quints (all-quint):", len(PAULI_YYYYY5))
print("  ZZZZZZ sexts (all-sext):", len(PAULI_ZZZZZZ6))
print("  XXXXXX sexts (all-sext):", len(PAULI_XXXXXX6))
print("  YYYYYY sexts (all-sext):", len(PAULI_YYYYYY6))

# 🔧 NEW: compute num_features_qrc directly from the Pauli lists
num_features_qrc = (
    len(PAULI_X) + len(PAULI_Y) + len(PAULI_Z) +
    len(PAULI_ZZ) + len(PAULI_XX) + len(PAULI_YY) +
    len(PAULI_ZZZ) + len(PAULI_XXX3) + len(PAULI_YYY3) +
    len(PAULI_ZZZZ4) + len(PAULI_XXXX4) + len(PAULI_YYYY4) +
    len(PAULI_ZZZZZ5) + len(PAULI_XXXXX5) + len(PAULI_YYYYY5) +
    len(PAULI_ZZZZZZ6) + len(PAULI_XXXXXX6) + len(PAULI_YYYYYY6)
)

print("Total QRC features per window:", num_features_qrc)

# 6. KERR-QRC FEATURE MAP (QUANTUM)

def apply_kerr_layer(qc, x, kerr_strength=KERR_STRENGTH):
    for q in range(NUM_QUBITS):
        theta_q = ANGLE_SCALE * x[q]
        kerr_angle = kerr_strength * (theta_q ** 2)
        qc.rz(kerr_angle, q)

def reservoir_features_kerr(input_window):
    T, F = input_window.shape
    qc = QuantumCircuit(NUM_QUBITS)

    for t in range(T):
        x = input_window[t]

        for q in range(NUM_QUBITS):
            qc.ry(ANGLE_SCALE * x[q], q)

        apply_kerr_layer(qc, x, kerr_strength=KERR_STRENGTH)

        theta_global = ANGLE_SCALE * x[8]
        for q in range(NUM_QUBITS):
            qc.rz(theta_global, q)

        qc.append(U_linear, range(NUM_QUBITS))

    state = Statevector.from_instruction(qc)

    feats = []

    for p in PAULI_X:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_Y:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_Z:
        feats.append(np.real(state.expectation_value(p)))

    for p in PAULI_ZZ:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_XX:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_YY:
        feats.append(np.real(state.expectation_value(p)))

    for p in PAULI_ZZZ:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_XXX3:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_YYY3:
        feats.append(np.real(state.expectation_value(p)))

    for p in PAULI_ZZZZ4:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_XXXX4:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_YYYY4:
        feats.append(np.real(state.expectation_value(p)))

    for p in PAULI_ZZZZZ5:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_XXXXX5:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_YYYYY5:
        feats.append(np.real(state.expectation_value(p)))

    for p in PAULI_ZZZZZZ6:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_XXXXXX6:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_YYYYYY6:
        feats.append(np.real(state.expectation_value(p)))

    feats = np.array(feats, dtype=float)
    assert feats.shape[0] == num_features_qrc, \
        f"Feature length {feats.shape[0]} != expected {num_features_qrc}"
    return feats

def build_reservoir_features_for_set_kerr(X_windows_scaled):
    num_samples_local = X_windows_scaled.shape[0]
    feat_list = []
    for idx, window in enumerate(X_windows_scaled):
        feat_list.append(reservoir_features_kerr(window))
        if (idx + 1) % 5 == 0:
            print(f"Processed {idx+1}/{num_samples_local} windows...", end="\r")
    print()
    return np.vstack(feat_list)

# 7. BUILD KERR-QRC FEATURE MATRIX

print("\nBuilding Kerr-QRC feature vectors (1–6 body)...")
QRC_features = build_reservoir_features_for_set_kerr(X_windows)
print("QRC_features shape:", QRC_features.shape)

assert QRC_features.shape[0] == num_samples
assert QRC_features.shape[1] == num_features_qrc

print("\nFeature means (first 5):", QRC_features.mean(axis=0)[:5])
print("Feature stds  (first 5):", QRC_features.std(axis=0)[:5])

# 8. TRAIN/TEST SPLIT FOR CLASSICAL READOUT

H = QRC_features

train_size = int(0.8 * num_samples)

X_train = H[:train_size]
X_test  = H[train_size:]

y_train_scaled = y_targets_scaled[:train_size]
y_test_scaled  = y_targets_scaled[train_size:]
y_train_raw    = y_targets_raw[:train_size]
y_test_raw     = y_targets_raw[train_size:]

print("\nTrain/Test shapes (QRC):")
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train_scaled:", y_train_scaled.shape)
print("y_test_scaled :", y_test_scaled.shape)
print("y_train_raw   :", y_train_raw.shape)
print("y_test_raw    :", y_test_raw.shape)

# 9. METRIC HELPER

def evaluate_model(y_true_raw, y_pred_raw, name="model"):
    mae = mean_absolute_error(y_true_raw, y_pred_raw)
    mse = mean_squared_error(y_true_raw, y_pred_raw)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true_raw, y_pred_raw)

    print(f"\n=== {name} ===")
    print("MAE  :", mae)
    print("RMSE :", rmse)
    print("R^2  :", r2)
    return r2

# 10. RIDGE REGRESSION ON FULL QRC FEATURES (ALPHA SWEEP)

alphas = [0.1, 1.0, 10.0, 100.0]
best_r2_full = -1e9
best_alpha_full = None

print(f"\nRidge on FULL {num_features_qrc}-dim QRC features (alpha sweep):")
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train_scaled)
    y_pred_scaled = ridge.predict(X_test)
    y_pred_raw = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
    r2 = evaluate_model(y_test_raw, y_pred_raw, name=f"Ridge (full QRC, alpha={alpha})")
    if r2 > best_r2_full:
        best_r2_full = r2
        best_alpha_full = alpha

print(f"\nBest Ridge on FULL QRC features: alpha={best_alpha_full}, R^2={best_r2_full:.4f}")

# 11. PCA + RIDGE (REDUCED QRC FEATURES)

pca_components = min(200, H.shape[1], H.shape[0] - 1)
print(f"\nRunning PCA with n_components = {pca_components}")

pca = PCA(n_components=pca_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca  = pca.transform(X_test)

print("X_train_pca shape:", X_train_pca.shape)
print("X_test_pca  shape:", X_test_pca.shape)


Columns in df: ['SETTLEMENT_PERIOD', 'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION', 'EMBEDDED_SOLAR_GENERATION', 'PUMP_STORAGE_PUMPING', 'SCOTTISH_TRANSFER', 'NEMO_FLOW', 'ELECLINK_FLOW', 'Temperature', 'period_sin', 'period_cos']
Number of rows before cleaning: 1488
Number of rows after dropna: 1488
X_raw shape: (1488, 9)
y_raw shape: (1488,)
X_scaled shape: (1488, 9)
y_scaled shape: (1488,)
X_windows shape       : (1480, 8, 9)
y_targets_scaled shape: (1480,)
y_targets_raw shape   : (1480,)
Built linear (Gaussian) unitary reservoir with DEPTH = 1
Pauli readout operators built:
  single X: 9
  single Y: 9
  single Z: 9
  ZZ pairs (all-pair): 36
  XX pairs (all-pair): 36
  YY pairs (all-pair): 36
  ZZZ triples (all-triple): 84
  XXX triples (all-triple): 84
  YYY triples (all-triple): 84
  ZZZZ quads  (all-quad): 126
  XXXX quads  (all-quad): 126
  YYYY quads  (all-quad): 126
  ZZZZZ quints (all-quint): 126
  XXXXX quints (all-quint): 126
  YYYYY quints (all-quint): 126
  ZZZZZZ se

## Large QRC (9 qubits, 1–6 body readout, 1 395 features)

In [21]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Pauli

FILEPATH = r"C:/Users/domin/OneDrive/Documents/QUAC 605 Project Data.xlsx"

np.random.seed(42)

NUM_QUBITS    = 9
SIGMA         = 0.1
DEPTH         = 1
WINDOW        = 8
ANGLE_SCALE   = 0.1
KERR_STRENGTH = 0.1

# 1. LOAD AND PREPARE RAW DATA (CLASSICAL)

df = pd.read_excel(FILEPATH)

print("Columns in df:", df.columns.tolist())
print("Number of rows before cleaning:", len(df))

max_period = 48
df["period_sin"] = np.sin(2 * np.pi * df["SETTLEMENT_PERIOD"] / max_period)
df["period_cos"] = np.cos(2 * np.pi * df["SETTLEMENT_PERIOD"] / max_period)

feature_cols = [
    "EMBEDDED_WIND_GENERATION",
    "EMBEDDED_SOLAR_GENERATION",
    "PUMP_STORAGE_PUMPING",
    "SCOTTISH_TRANSFER",
    "NEMO_FLOW",
    "ELECLINK_FLOW",
    "Temperature",
    "period_sin",
    "period_cos",
]

target_col = "ENGLAND_WALES_DEMAND"

df = df.dropna(subset=feature_cols + [target_col]).reset_index(drop=True)
print("Number of rows after dropna:", len(df))

X_raw = df[feature_cols].values.astype(float)
y_raw = df[target_col].values.astype(float)

print("X_raw shape:", X_raw.shape)
print("y_raw shape:", y_raw.shape)

# 2. SCALE INPUT FEATURES AND TARGET (CLASSICAL)

X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(X_raw)
print("X_scaled shape:", X_scaled.shape)

y_scaler = StandardScaler()
y_scaled = y_scaler.fit_transform(y_raw.reshape(-1, 1)).ravel()
print("y_scaled shape:", y_scaled.shape)

T_total, n_features = X_scaled.shape

# 3. BUILD SLIDING WINDOWS (CLASSICAL)

X_windows = []
y_targets_scaled = []
y_targets_raw    = []

for t in range(WINDOW, T_total):
    window = X_scaled[t - WINDOW : t]
    X_windows.append(window)
    y_targets_scaled.append(y_scaled[t])
    y_targets_raw.append(y_raw[t])

X_windows        = np.array(X_windows)
y_targets_scaled = np.array(y_targets_scaled)
y_targets_raw    = np.array(y_targets_raw)

num_samples = X_windows.shape[0]

print("X_windows shape       :", X_windows.shape)
print("y_targets_scaled shape:", y_targets_scaled.shape)
print("y_targets_raw shape   :", y_targets_raw.shape)

assert num_samples == y_targets_scaled.shape[0] == y_targets_raw.shape[0]

# 4. BUILD FIXED LINEAR RESERVOIR

def build_gaussian_unitary(num_qubits=NUM_QUBITS, depth=DEPTH, sigma=SIGMA):
    qc = QuantumCircuit(num_qubits)
    for _ in range(depth):
        for q in range(num_qubits):
            qc.rx(np.random.normal(0, sigma), q)
            qc.ry(np.random.normal(0, sigma), q)
            qc.rz(np.random.normal(0, sigma), q)
        for q in range(num_qubits - 1):
            qc.cz(q, q + 1)
    return qc.to_gate(label="U_linear")

U_linear = build_gaussian_unitary()
print("Built linear (Gaussian) unitary reservoir with DEPTH =", DEPTH)

# 5. PRE-BUILD PAULI OPERATORS FOR READOUT

def build_pauli_operators(num_qubits):
    single_X = []
    single_Y = []
    single_Z = []
    pair_ZZ  = []
    pair_XX  = []
    pair_YY  = []
    triple_ZZZ = []
    triple_XXX = []
    triple_YYY = []
    quad_ZZZZ  = []
    quad_XXXX  = []
    quad_YYYY  = []
    quint_ZZZZZ = []
    quint_XXXXX = []
    quint_YYYYY = []
    sext_ZZZZZZ = []
    sext_XXXXXX = []
    sext_YYYYYY = []

    for i in range(num_qubits):
        sX = "".join("X" if j == i else "I" for j in range(num_qubits))
        sY = "".join("Y" if j == i else "I" for j in range(num_qubits))
        sZ = "".join("Z" if j == i else "I" for j in range(num_qubits))
        single_X.append(Pauli(sX))
        single_Y.append(Pauli(sY))
        single_Z.append(Pauli(sZ))

    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            sZZ = ""
            sXX = ""
            sYY = ""
            for k in range(num_qubits):
                if k == i or k == j:
                    sZZ += "Z"
                    sXX += "X"
                    sYY += "Y"
                else:
                    sZZ += "I"
                    sXX += "I"
                    sYY += "I"
            pair_ZZ.append(Pauli(sZZ))
            pair_XX.append(Pauli(sXX))
            pair_YY.append(Pauli(sYY))

    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            for k in range(j + 1, num_qubits):
                sZZZ = ""
                sXXX = ""
                sYYY = ""
                for idx in range(num_qubits):
                    if idx == i or idx == j or idx == k:
                        sZZZ += "Z"
                        sXXX += "X"
                        sYYY += "Y"
                    else:
                        sZZZ += "I"
                        sXXX += "I"
                        sYYY += "I"
                triple_ZZZ.append(Pauli(sZZZ))
                triple_XXX.append(Pauli(sXXX))
                triple_YYY.append(Pauli(sYYY))

    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            for k in range(j + 1, num_qubits):
                for l in range(k + 1, num_qubits):
                    sZZZZ = ""
                    sXXXX = ""
                    sYYYY = ""
                    for idx in range(num_qubits):
                        if idx in (i, j, k, l):
                            sZZZZ += "Z"
                            sXXXX += "X"
                            sYYYY += "Y"
                        else:
                            sZZZZ += "I"
                            sXXXX += "I"
                            sYYYY += "I"
                    quad_ZZZZ.append(Pauli(sZZZZ))
                    quad_XXXX.append(Pauli(sXXXX))
                    quad_YYYY.append(Pauli(sYYYY))

    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            for k in range(j + 1, num_qubits):
                for l in range(k + 1, num_qubits):
                    for m in range(l + 1, num_qubits):
                        sZZZZZ = ""
                        sXXXXX = ""
                        sYYYYY = ""
                        for idx in range(num_qubits):
                            if idx in (i, j, k, l, m):
                                sZZZZZ += "Z"
                                sXXXXX += "X"
                                sYYYYY += "Y"
                            else:
                                sZZZZZ += "I"
                                sXXXXX += "I"
                                sYYYYY += "I"
                        quint_ZZZZZ.append(Pauli(sZZZZZ))
                        quint_XXXXX.append(Pauli(sXXXXX))
                        quint_YYYYY.append(Pauli(sYYYYY))

    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            for k in range(j + 1, num_qubits):
                for l in range(k + 1, num_qubits):
                    for m in range(l + 1, num_qubits):
                        for n in range(m + 1, num_qubits):
                            sZZZZZZ = ""
                            sXXXXXX = ""
                            sYYYYYY = ""
                            for idx in range(num_qubits):
                                if idx in (i, j, k, l, m, n):
                                    sZZZZZZ += "Z"
                                    sXXXXXX += "X"
                                    sYYYYYY += "Y"
                                else:
                                    sZZZZZZ += "I"
                                    sXXXXXX += "I"
                                    sYYYYYY += "I"
                            sext_ZZZZZZ.append(Pauli(sZZZZZZ))
                            sext_XXXXXX.append(Pauli(sXXXXXX))
                            sext_YYYYYY.append(Pauli(sYYYYYY))

    return (
        single_X, single_Y, single_Z,
        pair_ZZ, pair_XX, pair_YY,
        triple_ZZZ, triple_XXX, triple_YYY,
        quad_ZZZZ, quad_XXXX, quad_YYYY,
        quint_ZZZZZ, quint_XXXXX, quint_YYYYY,
        sext_ZZZZZZ, sext_XXXXXX, sext_YYYYYY
    )

(
    PAULI_X, PAULI_Y, PAULI_Z,
    PAULI_ZZ, PAULI_XX, PAULI_YY,
    PAULI_ZZZ, PAULI_XXX3, PAULI_YYY3,
    PAULI_ZZZZ4, PAULI_XXXX4, PAULI_YYYY4,
    PAULI_ZZZZZ5, PAULI_XXXXX5, PAULI_YYYYY5,
    PAULI_ZZZZZZ6, PAULI_XXXXXX6, PAULI_YYYYYY6
) = build_pauli_operators(NUM_QUBITS)

print("Pauli readout operators built:")
print("  single X:", len(PAULI_X))
print("  single Y:", len(PAULI_Y))
print("  single Z:", len(PAULI_Z))
print("  ZZ pairs (all-pair):", len(PAULI_ZZ))
print("  XX pairs (all-pair):", len(PAULI_XX))
print("  YY pairs (all-pair):", len(PAULI_YY))
print("  ZZZ triples (all-triple):", len(PAULI_ZZZ))
print("  XXX triples (all-triple):", len(PAULI_XXX3))
print("  YYY triples (all-triple):", len(PAULI_YYY3))
print("  ZZZZ quads  (all-quad):", len(PAULI_ZZZZ4))
print("  XXXX quads  (all-quad):", len(PAULI_XXXX4))
print("  YYYY quads  (all-quad):", len(PAULI_YYYY4))
print("  ZZZZZ quints (all-quint):", len(PAULI_ZZZZZ5))
print("  XXXXX quints (all-quint):", len(PAULI_XXXXX5))
print("  YYYYY quints (all-quint):", len(PAULI_YYYYY5))
print("  ZZZZZZ sexts (all-sext):", len(PAULI_ZZZZZZ6))
print("  XXXXXX sexts (all-sext):", len(PAULI_XXXXXX6))
print("  YYYYYY sexts (all-sext):", len(PAULI_YYYYYY6))

# 🔧 NEW: compute num_features_qrc directly from the Pauli lists
num_features_qrc = (
    len(PAULI_X) + len(PAULI_Y) + len(PAULI_Z) +
    len(PAULI_ZZ) + len(PAULI_XX) + len(PAULI_YY) +
    len(PAULI_ZZZ) + len(PAULI_XXX3) + len(PAULI_YYY3) +
    len(PAULI_ZZZZ4) + len(PAULI_XXXX4) + len(PAULI_YYYY4) +
    len(PAULI_ZZZZZ5) + len(PAULI_XXXXX5) + len(PAULI_YYYYY5) +
    len(PAULI_ZZZZZZ6) + len(PAULI_XXXXXX6) + len(PAULI_YYYYYY6)
)

print("Total QRC features per window:", num_features_qrc)

# 6. KERR-QRC FEATURE MAP (QUANTUM)

def apply_kerr_layer(qc, x, kerr_strength=KERR_STRENGTH):
    for q in range(NUM_QUBITS):
        theta_q = ANGLE_SCALE * x[q]
        kerr_angle = kerr_strength * (theta_q ** 2)
        qc.rz(kerr_angle, q)

def reservoir_features_kerr(input_window):
    T, F = input_window.shape
    qc = QuantumCircuit(NUM_QUBITS)

    for t in range(T):
        x = input_window[t]

        for q in range(NUM_QUBITS):
            qc.ry(ANGLE_SCALE * x[q], q)

        apply_kerr_layer(qc, x, kerr_strength=KERR_STRENGTH)

        theta_global = ANGLE_SCALE * x[8]
        for q in range(NUM_QUBITS):
            qc.rz(theta_global, q)

        qc.append(U_linear, range(NUM_QUBITS))

    state = Statevector.from_instruction(qc)

    feats = []

    for p in PAULI_X:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_Y:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_Z:
        feats.append(np.real(state.expectation_value(p)))

    for p in PAULI_ZZ:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_XX:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_YY:
        feats.append(np.real(state.expectation_value(p)))

    for p in PAULI_ZZZ:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_XXX3:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_YYY3:
        feats.append(np.real(state.expectation_value(p)))

    for p in PAULI_ZZZZ4:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_XXXX4:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_YYYY4:
        feats.append(np.real(state.expectation_value(p)))

    for p in PAULI_ZZZZZ5:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_XXXXX5:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_YYYYY5:
        feats.append(np.real(state.expectation_value(p)))

    for p in PAULI_ZZZZZZ6:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_XXXXXX6:
        feats.append(np.real(state.expectation_value(p)))
    for p in PAULI_YYYYYY6:
        feats.append(np.real(state.expectation_value(p)))

    feats = np.array(feats, dtype=float)
    assert feats.shape[0] == num_features_qrc, \
        f"Feature length {feats.shape[0]} != expected {num_features_qrc}"
    return feats

def build_reservoir_features_for_set_kerr(X_windows_scaled):
    num_samples_local = X_windows_scaled.shape[0]
    feat_list = []
    for idx, window in enumerate(X_windows_scaled):
        feat_list.append(reservoir_features_kerr(window))
        if (idx + 1) % 5 == 0:
            print(f"Processed {idx+1}/{num_samples_local} windows...", end="\r")
    print()
    return np.vstack(feat_list)

# 7. BUILD KERR-QRC FEATURE MATRIX

print("\nBuilding Kerr-QRC feature vectors (1–6 body)...")
QRC_features = build_reservoir_features_for_set_kerr(X_windows)
print("QRC_features shape:", QRC_features.shape)

assert QRC_features.shape[0] == num_samples
assert QRC_features.shape[1] == num_features_qrc

print("\nFeature means (first 5):", QRC_features.mean(axis=0)[:5])
print("Feature stds  (first 5):", QRC_features.std(axis=0)[:5])

# 8. TRAIN/TEST SPLIT FOR CLASSICAL READOUT

H = QRC_features

train_size = int(0.8 * num_samples)

X_train = H[:train_size]
X_test  = H[train_size:]

y_train_scaled = y_targets_scaled[:train_size]
y_test_scaled  = y_targets_scaled[train_size:]
y_train_raw    = y_targets_raw[:train_size]
y_test_raw     = y_targets_raw[train_size:]

print("\nTrain/Test shapes (QRC):")
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train_scaled:", y_train_scaled.shape)
print("y_test_scaled :", y_test_scaled.shape)
print("y_train_raw   :", y_train_raw.shape)
print("y_test_raw    :", y_test_raw.shape)

# 9. METRIC HELPER

def evaluate_model(y_true_raw, y_pred_raw, name="model"):
    mae = mean_absolute_error(y_true_raw, y_pred_raw)
    mse = mean_squared_error(y_true_raw, y_pred_raw)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true_raw, y_pred_raw)

    print(f"\n=== {name} ===")
    print("MAE  :", mae)
    print("RMSE :", rmse)
    print("R^2  :", r2)
    return r2

# 10. RIDGE REGRESSION ON FULL QRC FEATURES (ALPHA SWEEP)

alphas = [0.1, 1.0, 10.0, 100.0]
best_r2_full = -1e9
best_alpha_full = None

print(f"\nRidge on FULL {num_features_qrc}-dim QRC features (alpha sweep):")
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train_scaled)
    y_pred_scaled = ridge.predict(X_test)
    y_pred_raw = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
    r2 = evaluate_model(y_test_raw, y_pred_raw, name=f"Ridge (full QRC, alpha={alpha})")
    if r2 > best_r2_full:
        best_r2_full = r2
        best_alpha_full = alpha

print(f"\nBest Ridge on FULL QRC features: alpha={best_alpha_full}, R^2={best_r2_full:.4f}")

# 11. PCA + RIDGE (REDUCED QRC FEATURES)

pca_components = min(200, H.shape[1], H.shape[0] - 1)
print(f"\nRunning PCA with n_components = {pca_components}")

pca = PCA(n_components=pca_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca  = pca.transform(X_test)

print("X_train_pca shape:", X_train_pca.shape)
print("X_test_pca  shape:", X_test_pca.shape)


Columns in df: ['SETTLEMENT_PERIOD', 'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION', 'EMBEDDED_SOLAR_GENERATION', 'PUMP_STORAGE_PUMPING', 'SCOTTISH_TRANSFER', 'NEMO_FLOW', 'ELECLINK_FLOW', 'Temperature', 'period_sin', 'period_cos']
Number of rows before cleaning: 1488
Number of rows after dropna: 1488
X_raw shape: (1488, 9)
y_raw shape: (1488,)
X_scaled shape: (1488, 9)
y_scaled shape: (1488,)
X_windows shape       : (1480, 8, 9)
y_targets_scaled shape: (1480,)
y_targets_raw shape   : (1480,)
Built linear (Gaussian) unitary reservoir with DEPTH = 1
Pauli readout operators built:
  single X: 9
  single Y: 9
  single Z: 9
  ZZ pairs (all-pair): 36
  XX pairs (all-pair): 36
  YY pairs (all-pair): 36
  ZZZ triples (all-triple): 84
  XXX triples (all-triple): 84
  YYY triples (all-triple): 84
  ZZZZ quads  (all-quad): 126
  XXXX quads  (all-quad): 126
  YYYY quads  (all-quad): 126
  ZZZZZ quints (all-quint): 126
  XXXXX quints (all-quint): 126
  YYYYY quints (all-quint): 126
  ZZZZZZ se

In [22]:

# 11. PURELY CLASSICAL BASELINE: RANDOM FOREST ON FLATTENED CLASSICAL WINDOWS


def run_classical_rf_baseline(X_windows, y_targets_raw, train_fraction=0.8):
    
    num_samples_local = X_windows.shape[0]
    X_classical = X_windows.reshape(num_samples_local, -1)

    print("\n=== Classical RF Baseline (flattened windows) ===")
    print("X_classical shape:", X_classical.shape)

    
    train_size_local = int(train_fraction * num_samples_local)

    X_train_classical = X_classical[:train_size_local]
    X_test_classical  = X_classical[train_size_local:]

    y_train_classical = y_targets_raw[:train_size_local]   
    y_test_classical  = y_targets_raw[train_size_local:]   

    print("X_train_classical:", X_train_classical.shape)
    print("X_test_classical :", X_test_classical.shape)
    print("y_train_classical:", y_train_classical.shape)
    print("y_test_classical :", y_test_classical.shape)

    
    rf_classical = RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        random_state=42,
        n_jobs=-1
    )
    rf_classical.fit(X_train_classical, y_train_classical)

    
    y_pred_classical = rf_classical.predict(X_test_classical)

    mae = mean_absolute_error(y_test_classical, y_pred_classical)
    mse = mean_squared_error(y_test_classical, y_pred_classical)
    rmse = np.sqrt(mse)
    r2  = r2_score(y_test_classical, y_pred_classical)

    print("\n--- Classical RF Results ---")
    print("MAE :", mae)
    print("RMSE:", rmse)
    print("R^2 :", r2)

    return rf_classical, y_test_classical, y_pred_classical, r2


rf_classical_model, y_test_classical, y_pred_classical, r2_classical = \
  run_classical_rf_baseline(X_windows, y_targets_raw)


=== Classical RF Baseline (flattened windows) ===
X_classical shape: (1480, 72)
X_train_classical: (1184, 72)
X_test_classical : (296, 72)
y_train_classical: (1184,)
y_test_classical : (296,)

--- Classical RF Results ---
MAE : 958.3568243243243
RMSE: 1228.1288342704895
R^2 : 0.9448069562122159
